#Pull To View

In [1]:
import sqlite3
import pandas as pd

In [2]:
data_db = 'data.db'
view_db = 'view.db'

In [4]:
# While True:

# select row count from viewpoint and data

connV = sqlite3.connect(view_db)
df_dviewpoint = pd.read_sql("SELECT * from vViewpoint",connV, index_col = 'vViewpointId') # where dTable = dPrice
#vpricepos = df_dviewpoint.iloc[0,4] # need to add more where clauses
connV.close()
dPriceRow = df_dviewpoint['row'].values[0]


# select relvant data from D and write to V
connD = sqlite3.connect(data_db)
df_d2vprice = pd.read_sql("SELECT * from dPrice where dPriceId > %s" %dPriceRow, connD)
print df_d2vprice
connD.close()


if not df_d2vprice.empty:
    # in a cursor update the view and the counter

    prices = df_d2vprice[['iDate','symbol','roll_ave']]
    
    # get max id
    max_dPrice_id = df_d2vprice['dPriceId'].max()
    print 'max_dPrice_id', max_dPrice_id

    prices = prices.values.tolist()
 

    connV = sqlite3.connect(view_db)
    cV = connV.cursor()
    cV.executemany('INSERT INTO vPrice (date, symbol, price) VALUES (?,?,?)', prices)
    cV.execute("UPDATE vViewpoint SET row = %s WHERE dtable = 'dPrice'" %max_dPrice_id) #)) #, 'dPrice'))
    for row in cV.execute('SELECT * FROM vPrice'):
        print row

    for row in cV.execute('SELECT * FROM vViewpoint'):
        print row

    connV.commit()
    connV.close()



   dPriceId  iPriceId                       iDate                  pStartDate  \
0       105       105  2015-06-03 19:01:21.122329  2015-06-03 19:05:00.022431   
1       106       106  2015-06-03 19:01:23.374512  2015-06-03 19:05:00.071585   
2       107       107  2015-06-03 19:01:25.577699  2015-06-03 19:05:00.121283   
3       108       108  2015-06-03 19:01:27.767089  2015-06-03 19:05:00.168860   
4       109       109  2015-06-03 19:01:30.417573  2015-06-03 19:05:00.219121   
5       110       110  2015-06-03 19:04:36.932651  2015-06-03 19:05:00.271225   
6       111       111  2015-06-03 19:04:40.837993  2015-06-03 19:05:00.322440   
7       112       112  2015-06-03 19:04:43.015796  2015-06-03 19:05:00.371197   
8       113       113  2015-06-03 19:04:45.205543  2015-06-03 19:05:00.420500   

                     pEndDate                          date  symbol     bid  \
0  2015-06-03 19:05:00.060744  2015-06-03 23:01:00 UTC+0000  GBPUSD  1.5317   
1  2015-06-03 19:05:00.109915  